In [31]:
from collections import defaultdict 
#Class to represent a graph 
class Graph: 
  
    def __init__(self, vertices): 
        self.V = vertices 

    # Floyd Warshall algorithm 
    def transitiveClosure(self,graph): 
        '''reach[][] will be the output matrix that will finally 
        have reachability values. 
        Initialize the solution matrix same as input graph matrix'''
        reach =[i[:] for i in graph] 
        '''Add all vertices one by one to the set of intermediate 
        vertices. 
         ---> Before start of a iteration, we have reachability value 
         for all pairs of vertices such that the reachability values 
          consider only the vertices in set  
        {0, 1, 2, .. k-1} as intermediate vertices. 
          ----> After the end of an iteration, vertex no. k is 
         added to the set of intermediate vertices and the  
        set becomes {0, 1, 2, .. k}'''
        for k in range(self.V): 
              
            # Pick all vertices as source one by one 
            for i in range(self.V): 
                  
                # Pick all vertices as destination for the 
                # above picked source 
                for j in range(self.V): 
                      
                    # If vertex k is on a path from i to j,  
                       # then make sure that the value of reach[i][j] is 1 
                    reach[i][j] = min( reach[i][j] , reach[i][k] + reach[k][j] )
        return reach

In [53]:
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import tqdm
import numpy as np

num_PC = 0
num_CP = 0
num_CO = 0
num_NO = 0

def relation_getter(i, j, graph, num_CO, num_PC, num_CP, num_NO):
    if graph[i][j] < 100 or graph[j][i] < 100:
        num_CO += 1
        return "Coref", num_CO, num_PC, num_CP, num_NO
    elif graph[i][j] < 10000:
        num_PC += 1
        return "SuperSub", num_CO, num_PC, num_CP, num_NO
    elif graph[j][i] < 10000:
        num_CP += 1
        return "SubSuper", num_CO, num_PC, num_CP, num_NO
    else:
        num_NO += 1
        return "NoRel", num_CO, num_PC, num_CP, num_NO

mypath = "LDC2016E47_IC_Domain_Event_Annotation_From_CMU_V1.0/data/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and f[-3:] == "xml"]
print(len(onlyfiles))

#fname = "AFP_ENG_19950125.0289.src.xml.txt.blk.tok.stp.tbf.xml"

for fname in tqdm.tqdm(onlyfiles):
    tree = ET.parse(mypath+fname)
    root = tree.getroot()
    Text = ""
    event_dict = {}
    eventid2num = {}
    relation_dict = {}
    num = 1
    last_eventid = ''
    relations = ['coreference', 'subevent_of', 'in_reporting', 'member_of']
    with open('./IC_Processed/' + fname + '.tsvx', 'w', encoding='utf8') as fp:
        for sentence in root:
            for word in sentence:
                if word.get('wd'):
                    start = len(Text)
                    Text += word.get('wd') + " "
                    if word.get('eventid') and last_eventid != word.get('eventid'):
                        if word.get('event_type'):
                            event_type = word.get('event_type')
                        else:
                            event_type = 'event'
                        eventid2num[word.get('eventid')] = num
                        event_dict[num] = {'eventid': str(num), 'mention': word.get('wd'), 'event_type': event_type, 'char_ID': start}
                        num += 1
                        last_eventid = word.get('eventid')
                    for relation in relations:
                        if word.get(relation):
                            if word.get(relation).find('+'):
                                eventid_list = word.get(relation).split('+')
                                for eventid in eventid_list:
                                    relation_dict[(word.get('eventid'), eventid)] = relation
                            else:
                                relation_dict[(word.get('eventid'), word.get(relation))] = relation
            Text += ". "

        fp.write('Text\t' + Text + '\n')
        for key, value in event_dict.items():
            fp.write('Event\t')
            fp.write(value['eventid'] + "\t")
            fp.write(value['mention'] + "\t")
            fp.write(value['event_type'] + "\t")
            fp.write(str(value['char_ID']) + "\n")
            
        relation_dict_fixed = {}
        for key, value in relation_dict.items():
            try:
                relation_dict_fixed[(eventid2num[key[0]], eventid2num[key[1]])] = value
            except:
                print(fname)
                print(key[0])
                print(key[1])
            
        g = Graph(num)
        graph = np.ones((num, num))
        graph = graph * 10000
        for edge, rel in relation_dict_fixed.items():
            if rel == "coreference":
                graph[int(edge[0])][int(edge[1])] = 1
                graph[int(edge[1])][int(edge[0])] = 1
            if rel in ['subevent_of', 'member_of']:
                graph[int(edge[1])][int(edge[0])] = 100
        graph = g.transitiveClosure(graph) 
        for i in range(1, num):
            for j in range(i+1, num):
                if i < j:
                    relation, num_CO, num_PC, num_CP, num_NO = relation_getter(int(event_dict[i]['eventid']), int(event_dict[j]['eventid']), graph, num_CO, num_PC, num_CP, num_NO)
                    fp.write('Relation\t')
                    fp.write(event_dict[i]['eventid'] + "\t")
                    fp.write(event_dict[j]['eventid'] + "\t")
                    fp.write(relation + "\t")
                    fp.write("true\t")
                    fp.write(event_dict[i]["mention"] + "\t")
                    fp.write(event_dict[j]["mention"] + "\n")
                
print("num_PC:", num_PC)     
print("num_CP:", num_CP)     
print("num_CO:", num_CO)     
print("num_NO:", num_NO)

  1%|          | 1/100 [00:00<00:11,  8.54it/s]

100
APW_ENG_19961217.1333.src.xml.txt.blk.tok.stp.tbf.xml
19
27
APW_ENG_19961217.1333.src.xml.txt.blk.tok.stp.tbf.xml
24
22
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
12
45
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
13
45
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
15
45
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
17
45
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
20
19
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
22
19
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
26
25
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
27
25
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
29
28
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
30
28
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
34
33
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
35
33
APW_ENG_19970521.1259.src.xml.txt.blk.tok.stp.tbf.xml
37
33


  5%|▌         | 5/100 [00:00<00:08, 11.45it/s]

XIN_ENG_19960116.0110.src.xml.txt.blk.tok.stp.tbf.xml
16
15
XIN_ENG_19960116.0110.src.xml.txt.blk.tok.stp.tbf.xml
17
15
AFP_ENG_20030303.0023.src.xml.txt.blk.tok.stp.tbf.xml
4
2
AFP_ENG_20040813.0336.src.xml.txt.blk.tok.stp.tbf.xml
7
55
AFP_ENG_20040813.0336.src.xml.txt.blk.tok.stp.tbf.xml
13
57
AFP_ENG_20040813.0336.src.xml.txt.blk.tok.stp.tbf.xml
19
57
AFP_ENG_20040813.0336.src.xml.txt.blk.tok.stp.tbf.xml
20
57
AFP_ENG_20040813.0336.src.xml.txt.blk.tok.stp.tbf.xml
30
29
AFP_ENG_20040813.0336.src.xml.txt.blk.tok.stp.tbf.xml
38
57
AFP_ENG_20040813.0336.src.xml.txt.blk.tok.stp.tbf.xml
47
57
APW_ENG_19970713.0540.src.xml.txt.blk.tok.stp.tbf.xml
11
85
APW_ENG_19970713.0540.src.xml.txt.blk.tok.stp.tbf.xml
15
85
APW_ENG_19970713.0540.src.xml.txt.blk.tok.stp.tbf.xml
34
85
APW_ENG_19970713.0540.src.xml.txt.blk.tok.stp.tbf.xml
97
90


  9%|▉         | 9/100 [00:00<00:09,  9.39it/s]

...

XIN_ENG_20030919.0267.src.xml.txt.blk.tok.stp.tbf.xml
1
12
XIN_ENG_20030919.0267.src.xml.txt.blk.tok.stp.tbf.xml
13
12
XIN_ENG_20030919.0267.src.xml.txt.blk.tok.stp.tbf.xml
48
47
XIN_ENG_20030919.0267.src.xml.txt.blk.tok.stp.tbf.xml
50
47
XIN_ENG_20030919.0267.src.xml.txt.blk.tok.stp.tbf.xml
51
47


 98%|█████████▊| 98/100 [00:11<00:00,  9.21it/s]

NYT_ENG_20011116.0315.src.xml.txt.blk.tok.stp.tbf.xml
24
21
AFP_ENG_19950407.0079.src.xml.txt.blk.tok.stp.tbf.xml
31
30
AFP_ENG_20020724.0397.src.xml.txt.blk.tok.stp.tbf.xml
31
30


100%|██████████| 100/100 [00:11<00:00,  8.35it/s]

num_PC: 2248
num_CP: 2338
num_CO: 2353
num_NO: 81887


In [51]:
%debug

> <ipython-input-50-8fdc7416698e>(97)<module>()
     95             for j in range(i+1, num+1):
     96                 if i < j:
---> 97                     relation, num_CO, num_PC, num_CP, num_NO = relation_getter(int(event_dict[i]['eventid']), int(event_dict[j]['eventid']), graph, num_CO, num_PC, num_CP, num_NO)
     98                     fp.write('Relation\t')
     99                     fp.write(event_dict[i]['eventid'] + "\t")

ipdb> event_dict[i]['eventid']
'1'
ipdb> event_dict[j]['eventid']
*** KeyError: 23
ipdb> j
*** The 'jump' command requires a line number
ipdb> num
23
ipdb> event_dict
{1: {'eventid': '1', 'mention': 'take', 'event_type': 'event', 'char_ID': 10}, 2: {'eventid': '2', 'mention': 'attacked', 'event_type': 'event', 'char_ID': 103}, 3: {'eventid': '3', 'mention': 'took', 'event_type': 'event', 'char_ID': 160}, 4: {'eventid': '4', 'mention': 'said', 'event_type': 'report', 'char_ID': 276}, 5: {'eventid': '5', 'mention': 'set', 'event_type': 'event', 'char_ID': 